In [1]:
import os
import glob
import numpy as np
import xarray as xr
import rioxarray
from pyproj import Geod
import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import folium
from folium import plugins
import geopandas as gpd
from shapely.geometry import Polygon
import json

dataset_dir = f"{os.path.dirname(os.getcwd())}/datasets"

In [2]:
# Load ocean current data
current_files = sorted(glob.glob(f"{dataset_dir}/realtime_surface_ocean_currents/oscar_currents_nrt_*.nc"))
print(f"Found {len(current_files)} current data files")

# Open multi-file dataset
currents_ds = xr.open_mfdataset(current_files, engine="netcdf4", combine='nested', concat_dim='time')

# Set spatial reference using rioxarray (EPSG:4326 = WGS84)
currents_ds.rio.write_crs("EPSG:4326", inplace=True)

# Rename coordinates to standard names for consistency
if 'lon' in currents_ds.coords and 'lat' in currents_ds.coords:
    currents_ds = currents_ds.rename({'lon': 'longitude', 'lat': 'latitude'})

print(f"Time range: {currents_ds['time'].values[0]} to {currents_ds['time'].values[-1]}")
print(f"Spatial extent: lon [{currents_ds.longitude.min().values:.2f}, {currents_ds.longitude.max().values:.2f}], "
      f"lat [{currents_ds.latitude.min().values:.2f}, {currents_ds.latitude.max().values:.2f}]")
print(f"Shape: {currents_ds['ug'].shape}")
currents_ds

Found 10 current data files
Time range: 2025-09-22 00:00:00 to 2025-10-01 00:00:00
Spatial extent: lon [0.00, 359.75], lat [-89.75, 89.75]
Shape: (10, 1440, 719)


/tmp/ipykernel_125960/4220538050.py:13: UserWarning: rename 'lon' to 'longitude' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  currents_ds = currents_ds.rename({'lon': 'longitude', 'lat': 'latitude'})
/tmp/ipykernel_125960/4220538050.py:13: UserWarning: rename 'lat' to 'latitude' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  currents_ds = currents_ds.rename({'lon': 'longitude', 'lat': 'latitude'})


<xarray.Dataset> Size: 331MB
Dimensions:      (time: 10, longitude: 1440, latitude: 719)
Coordinates:
    latitude     (latitude) float64 6kB dask.array<chunksize=(719,), meta=np.ndarray>
    longitude    (longitude) float64 12kB dask.array<chunksize=(1440,), meta=np.ndarray>
  * time         (time) object 80B 2025-09-22 00:00:00 ... 2025-10-01 00:00:00
    spatial_ref  int64 8B 0
Data variables:
    u            (time, longitude, latitude) float64 83MB dask.array<chunksize=(1, 720, 360), meta=np.ndarray>
    v            (time, longitude, latitude) float64 83MB dask.array<chunksize=(1, 720, 360), meta=np.ndarray>
    ug           (time, longitude, latitude) float64 83MB dask.array<chunksize=(1, 720, 360), meta=np.ndarray>
    vg           (time, longitude, latitude) float64 83MB dask.array<chunksize=(1, 720, 360), meta=np.ndarray>
Attributes: (12/36)
    title:                      Ocean Surface Current Analyses Real-time (OSC...
    summary:                    Global, daily, 0.25 degree geostrophic and to...
    keywords:                   ocean currents,ocean circulation,surface curr...
    Conventions:                CF-1.8 Standard Names v77, ACDD-1.3, netcdf 4...
    id:                         OSCAR_L4_OC_NRT_V2.0
    history:                    OSCAR 0.25 degree daily version 2.0 replaces ...
    ...                         ...
    geospatial_lon_max:         359.75
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.25 degree
    time_coverage_start:        2025-09-22T00:00:00
    time_coverage_end:          2025-09-22T23:59:59
    date_created:               2025-10-02

In [3]:
# Convert OSCAR NetCDF to GRIB2 using eccodes
import xarray as xr
import numpy as np
from eccodes import *

# Load data
ds = xr.open_dataset(
    "/home/luke/Code/wolfram-nasa-hackathon-2025/datasets/realtime_surface_ocean_currents/oscar_currents_nrt_20251001.nc"
)
u = ds['ug'].isel(time=0)
v = ds['vg'].isel(time=0)

# Check dimension and coordinate names
print(f"Dimensions: {u.dims}")
print(f"Shape: {u.shape}")
print(f"Coordinates: {list(u.coords.keys())}")

# Get lon/lat arrays
lon_vals = u.lon.values if 'lon' in u.coords else u.longitude.values
lat_vals = u.lat.values if 'lat' in u.coords else u.latitude.values

# Shift longitudes from 0-360 to -180-180
lon_shifted = (lon_vals + 180) % 360 - 180
sort_idx = np.argsort(lon_shifted)
lon_shifted = lon_shifted[sort_idx]

# Reorder data - check if it's (lon, lat) or (lat, lon)
if u.dims[0] in ['lon', 'longitude']:
    # Data is (lon, lat)
    u_data = u.values[sort_idx, :]
    v_data = v.values[sort_idx, :]
else:
    # Data is (lat, lon)
    u_data = u.values[:, sort_idx]
    v_data = v.values[:, sort_idx]

print(f"Shifted lon range: {lon_shifted.min():.2f} to {lon_shifted.max():.2f}")
print(f"Data shape after reordering: {u_data.shape}")

# Flip data vertically for leaflet-velocity (needs north-to-south)
u_data_flipped = np.flipud(u_data.T).T 
v_data_flipped = np.flipud(v_data.T).T
lat_vals_reversed = lat_vals[::-1]
print(f"New lat range: {lat_vals_reversed[0]:.2f} to {lat_vals_reversed[-1]:.2f}")

# Write GRIB2 file
with open("ocean_velocity_new.grb2", "wb") as f:
    for name, data, param_num in [("u", u_data_flipped, 2), ("v", v_data_flipped, 3)]:
        gid = codes_grib_new_from_samples("regular_ll_sfc_grib2")
        codes_set(gid, "editionNumber", 2)
        codes_set(gid, "centre", 74)
        codes_set(gid, "significanceOfReferenceTime", 3)
        codes_set(gid, "year", 2025)
        codes_set(gid, "month", 10)
        codes_set(gid, "day", 1)
        codes_set(gid, "discipline", 10)
        codes_set(gid, "parameterCategory", 1)
        codes_set(gid, "parameterNumber", param_num)
        codes_set(gid, "typeOfFirstFixedSurface", 1)
        codes_set(gid, "Ni", len(lon_shifted))
        codes_set(gid, "Nj", len(lat_vals_reversed))
        codes_set(gid, "latitudeOfFirstGridPointInDegrees", float(lat_vals_reversed[0]))
        codes_set(gid, "latitudeOfLastGridPointInDegrees", float(lat_vals_reversed[-1]))
        codes_set(gid, "longitudeOfFirstGridPointInDegrees", float(lon_shifted[0]))
        codes_set(gid, "longitudeOfLastGridPointInDegrees", float(lon_shifted[-1]))
        codes_set(gid, "iDirectionIncrementInDegrees", 0.25)
        codes_set(gid, "jDirectionIncrementInDegrees", 0.25)
        codes_set(gid, "scanningMode", 0)
        values = data.T.flatten()  # Transpose to (lat, lon) then flatten
        values = np.where(np.isnan(values), -9999.0, values)
        codes_set(gid, "bitmapPresent", 1)
        codes_set(gid, "missingValue", -9999.0)
        codes_set_double_array(gid, "values", values)
        codes_write(gid, f)
        codes_release(gid)
print(f"   Shape: ({len(lat_vals_reversed)}, {len(lon_shifted)})")
print(f"   Coordinates: lon [{lon_shifted[0]:.1f}, {lon_shifted[-1]:.1f}], lat [{lat_vals_reversed[0]:.1f}, {lat_vals_reversed[-1]:.1f}]")

PRE-MAIN-DEBUG Registering library [eckit] with address [0x7fe767caf060]
PRE-MAIN-DEBUG Registering library [eckit_geo] with address [0x7fe767fb99e0]
PRE-MAIN-DEBUG Registering http resource [/agent] to registry with address [0x7fe768fd5920]
PRE-MAIN-DEBUG Registering http resource [/cgi] to registry with address [0x7fe768fd5dc0]
PRE-MAIN-DEBUG Registering http resource [/config] to registry with address [0x7fe768fd5e20]
PRE-MAIN-DEBUG Registering http resource [/files] to registry with address [0x7fe768fd5ee0]
PRE-MAIN-DEBUG Registering http resource [/image] to registry with address [0x7fe768fd5f80]
PRE-MAIN-DEBUG Registering http resource [/html] to registry with address [0x7fe768fd5f40]
PRE-MAIN-DEBUG Registering http resource [/java] to registry with address [0x7fe768fd6060]
PRE-MAIN-DEBUG Registering library [fckit] with address [0x7fe766dfbf40]
Dimensions: ('longitude', 'latitude')
Shape: (1440, 719)
Coordinates: ['lat', 'lon', 'time']
Shifted lon range: -180.00 to 179.75
Data s

In [4]:
# Convert GRIB2 to JSON and fix for leaflet-velocity
import subprocess
import json

# Run grib2json
print("Converting GRIB2 to JSON...")
result = subprocess.run([
    './grib2json/target/grib2json-0.8.0-SNAPSHOT/bin/grib2json',
    '--data',
    '--output', 'ocean_velocity.json',
    '--names',
    '--compact',
    'ocean_velocity_new.grb2'
], capture_output=True, text=True)

if result.returncode != 0:
    print(f"Error: {result.stderr}")
else:    
    # Fix "NaN" strings to null for proper JSON/JavaScript handling
    with open('ocean_velocity.json', 'r') as f:
        data = json.load(f)
    
    # Convert "NaN" strings to null
    for record in data:
        record['data'] = [None if v == "NaN" else v for v in record['data']]
    
    with open('ocean_velocity.json', 'w') as f:
        json.dump(data, f, separators=(',', ':'))

Converting GRIB2 to JSON...


## Mesoscale Eddy Detection using Okubo-Weiss Parameter

The Okubo-Weiss parameter identifies coherent vortex structures (eddies) in the flow field:
- **W < 0**: Vorticity-dominated regions (eddies)
- **W > 0**: Strain-dominated regions (stretching/shearing flow)

In [5]:
def calculate_spatial_derivatives(ds, u_var='ug', v_var='vg', time_idx=0):
    """Calculate spatial derivatives of velocity components."""
    # Select time slice
    u = ds[u_var].isel(time=time_idx)
    v = ds[v_var].isel(time=time_idx)
    lons = ds['longitude'].values
    lats = ds['latitude'].values
    R = 6371000  # Earth radius in meters
    dlon_deg = np.mean(np.gradient(lons))  # longitude spacing in degrees (constant)
    dlat_deg = np.mean(np.gradient(lats))  # latitude spacing in degrees (constant)
    dx_val = R * np.cos(np.radians(lats)) * (dlon_deg * np.pi / 180)
    dx = np.outer(np.ones(len(lons)), dx_val)  # shape (nlon, nlat)
    dy_val = R * (dlat_deg * np.pi / 180)
    dy = np.full((len(lons), len(lats)), dy_val)  # shape (nlon, nlat)

    du_dx = np.gradient(u.values, axis=0) / dx
    du_dy = np.gradient(u.values, axis=1) / dy
    dv_dx = np.gradient(v.values, axis=0) / dx
    dv_dy = np.gradient(v.values, axis=1) / dy

    return du_dx, du_dy, dv_dx, dv_dy, lons, lats

### Calculate velocity gradients and strain/vorticity fields

In [6]:
# Calculate spatial derivatives for the latest time step
du_dx, du_dy, dv_dx, dv_dy, lons, lats = calculate_spatial_derivatives(currents_ds, time_idx=-1)

# Calculate strain rate components
S_n = du_dx - dv_dy  # Normal strain rate (s^-1)
S_s = dv_dx + du_dy  # Shear strain rate (s^-1)

# Calculate vorticity
zeta = dv_dx - du_dy  # Vertical vorticity (s^-1)

# Okubo-Weiss parameter
W = S_n**2 + S_s**2 - zeta**2

# Create xarray DataArrays
# okubo_weiss = xr.DataArray(
#     W,
#     coords={'longitude': lons, 'latitude': lats},
#     dims=['longitude', 'latitude'],
#     name='okubo_weiss',
#     attrs={
#         'long_name': 'Okubo-Weiss parameter',
#         'units': 's^-2',
#         'description': 'W < 0: vorticity-dominated (eddies), W > 0: strain-dominated'
#     }
# )

vorticity = xr.DataArray(
    zeta,
    coords={'longitude': lons, 'latitude': lats},
    dims=['longitude', 'latitude'],
    name='vorticity',
    attrs={'long_name': 'Vertical vorticity', 'units': 's^-1'}
)
# okubo_weiss

### Visualize eddy fields for latest time step

In [7]:
# # Visualize vorticity field with smooth contours
# fig = plt.figure(figsize=(16, 8))
# ax = plt.axes(projection=ccrs.PlateCarree())

# # Use filled contours for smooth blob visualization
# vort_data = vorticity.values.T
# levels = np.linspace(-5e-6, 5e-6, 5)
# cf = ax.contourf(
#     lons, lats, vort_data,
#     levels=levels,
#     cmap='coolwarm',
#     transform=ccrs.PlateCarree(),
#     extend='both'
# )

# ax.coastlines(linewidth=0.5)
# ax.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.5)
# ax.add_feature(cfeature.BORDERS, linewidth=0.3, alpha=0.5)
# ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)

# cbar = plt.colorbar(cf, ax=ax, shrink=0.8, pad=0.05)
# cbar.set_label('Vertical Vorticity (s⁻¹)', fontsize=10)
# ax.set_title('Vertical Vorticity Field',
#              fontsize=12, fontweight='bold')
# plt.tight_layout()
# plt.show()

In [8]:
# # Streamplot of velocity field
# time_idx = -1  # latest time step
# u_data = currents_ds['ug'].isel(time=time_idx).values.T
# v_data = currents_ds['vg'].isel(time=time_idx).values.T

# speed = np.sqrt(u_data**2 + v_data**2)

# fig = plt.figure(figsize=(16, 8))
# ax = plt.axes(projection=ccrs.PlateCarree())

# stream = ax.streamplot(
#     lons, lats, u_data, v_data,
#     color=speed,
#     cmap='viridis',
#     density=3,
#     linewidth=1.0,
#     arrowsize=1.0,
#     transform=ccrs.PlateCarree()
# )

# ax.coastlines(linewidth=0.5)
# ax.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.7)
# ax.add_feature(cfeature.BORDERS, linewidth=0.3, alpha=0.5)
# ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)

# cbar = plt.colorbar(stream.lines, ax=ax, shrink=0.8, pad=0.05)
# cbar.set_label('Current Speed (m/s)', fontsize=10)
# ax.set_title(f'Ocean Surface Current Streamlines',
#              fontsize=12, fontweight='bold')

# plt.tight_layout()
# plt.show()

Okubo Weiss didn't seem much better than vorticity, so I think we leave it out.

In [ ]:
# Export vorticity data as JSON for React frontend
vorticity_json = []
for i, lon in enumerate(lons):
    for j, lat in enumerate(lats):
        val = vorticity.values[i, j]
        if not np.isnan(val):
            # Convert lon from 0-360 to -180-180
            adjusted_lon = lon if lon <= 180 else lon - 360
            vorticity_json.append({
                "lat": float(lat),
                "lng": float(adjusted_lon),
                "value": float(val)
            })

# Save to JSON file for frontend
import json
with open('../frontend/public/vorticity.json', 'w') as f:
    json.dump(vorticity_json, f)

print(f"Exported {len(vorticity_json)} vorticity points to vorticity.json")

# Also create the Folium map visualization
from folium.plugins import HeatMap

sample_factor = 1
vort_sampled = vorticity.values[::sample_factor, ::sample_factor]
lons_sampled = lons[::sample_factor]
lats_sampled = lats[::sample_factor]

m = folium.Map(location=[0, 180], zoom_start=2, tiles='CartoDB dark_matter')

from branca.colormap import LinearColormap
vmin, vmax = -1e-6, 1e-6

colors = ['#0d0887', '#5302a3', '#8b0aa5', '#b93289', '#db5c68', 
          '#f48849', '#febd2a', '#f0f921']
colormap = LinearColormap(colors=colors, vmin=vmin, vmax=vmax, 
                          caption='Vertical Vorticity (s⁻¹)')

heat_data = []
for i, lon in enumerate(lons_sampled):
    for j, lat in enumerate(lats_sampled):
        val = vort_sampled[i, j]
        if not np.isnan(val):
            # Normalize to 0-1 for heatmap
            normalized = (val - vmin) / (vmax - vmin)
            heat_data.append([lat, lon if lon <= 180 else lon - 360, normalized])
HeatMap(heat_data, min_opacity=0.03, radius=8, blur=4, 
        gradient={0.0: '#0d0887', 0.25: "#111011", 0.5: '#cc4778', 
                  0.75: '#f89540', 1.0: '#f0f921'}).add_to(m)
colormap.add_to(m)

output_path = f"./vorticity_map.html"
m.save(output_path)
m

In [10]:
# Export vorticity data for React frontend (leaflet.heat format)
import json

# Prepare heatmap data in [lat, lng, intensity] format
vmin, vmax = -1e-6, 1e-6
heat_data = []

for i, lon in enumerate(lons):
    for j, lat in enumerate(lats):
        val = vorticity.values[i, j]
        if not np.isnan(val):
            # Normalize to 0-1 for heatmap intensity
            normalized = (val - vmin) / (vmax - vmin)
            normalized = max(0, min(1, normalized))  # Clamp to [0, 1]
            
            # Convert lon from 0-360 to -180-180
            adjusted_lon = float(lon if lon <= 180 else lon - 360)
            
            heat_data.append([float(lat), adjusted_lon, float(normalized)])

print(f"Generated {len(heat_data)} vorticity points")

# Save to frontend
output_path = '../frontend/src/pages/landing/vorticity_data.json'
with open(output_path, 'w') as f:
    json.dump(heat_data, f)

print(f"Saved to {output_path}")

Generated 657305 vorticity points
Saved to ../frontend/src/pages/landing/vorticity_data.json
